### (준비단계) https://developer.riotgames.com/ 에서 로그인 후<br/><br/> 우측 상단 계정 아이콘 클릭, DASHBOARD에서 API KEY 발급받기

### 라이엇 게임즈에서 받은 API 키를 입력하세요

In [1]:
api_key = "RGAPI-b5a68708-0987-4621-9f1d-76f0a4f48d9e" # 라이엇 계정
# Expires: Tue, Nov 10th, 2020 @ 5:06pm (PT).대한민국 시간 기준 11월 11일 10:06까지 유효
# 대한민국 시간은 PT보다 17시간 더 빠릅니다.

### 라이브러리 Import

In [2]:
import pandas as pd
import numpy as np
import requests
import pickle
import time

### (1단계) 원하는 티어의 소환사 id 수집: 티어, 소환사 이름 얻기<br/><br/>(2단계) 소환사 닉네임을 이용해 각 소환사의 고유 ID(accountId) 수집<br/><br/>(3단계) 고유 ID를 이용해 게임 ID(매치 ID) 얻기

In [3]:
# (1단계) 원하는 티어의 소환사 id 수집: 티어, 소환사 이름 얻기
leagueQueue = 'RANKED_SOLO_5x5'
targetTiers = ['GOLD']
# 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND'
divisions = ['III', 'II', 'I']
pages = [2, 3, 4, 5]
# pages = [2, 3, 4, 5]

for page in pages:
    for tier in targetTiers:
        for division in divisions:
            targetSummoners = 'https://kr.api.riotgames.com/lol/league/v4/entries/'\
                            +leagueQueue+'/'+tier+'/'+division+'?page={}&api_key='+api_key
            targetSummonersInfo = requests.get(targetSummoners.format(page))

            while targetSummonersInfo.status_code != 200:
                time.sleep(5)
                targetSummoners = 'https://kr.api.riotgames.com/lol/league/v4/entries/'\
                                +leagueQueue+'/'+tier+'/'+division+'?page={}&api_key='+api_key
                targetSummonersInfo = requests.get(targetSummoners.format(page))

            targetSummonersInfo_df = pd.DataFrame(targetSummonersInfo.json())
            targetSummonersInfo_df = targetSummonersInfo_df.drop(['leagueId'], axis=1)
            print(tier, division, 'start')

    # (2단계) 소환사 닉네임을 이용해 각 소환사의 고유 ID(accountId) 수집
            accountIdList = []
            summonerNameList = targetSummonersInfo_df['summonerName'].tolist()
            print('summonerNameList', len(summonerNameList))
            for i in range(len(summonerNameList)):
                sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'\
                            + summonerNameList[i] + '?api_key=' + api_key
                r = requests.get(sohwan)

                start_time = time.time()
                while r.status_code != 200:
                    if time.time() - start_time >= 30:
                        print("During get {}th summonerName's accountID, problem has been raised".format(i))
                        break
                    time.sleep(5)
                    sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'\
                            + summonerNameList[i] + '?api_key=' + api_key 
                    r = requests.get(sohwan)

                if r.status_code == 200:
                    accountIdList.append(r.json()['accountId'])

    # (3단계) 고유 ID를 이용해 게임 ID(매치 ID) 얻기
            match_info_df = pd.DataFrame()
            print('accountIdList', len(accountIdList))
            for i in range(len(accountIdList)):
                match_url = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/'\
                            + accountIdList[i]+ '?queue=420&season=13' + '&api_key=' + api_key
                r = requests.get(match_url)

                start_time = time.time()
                while r.status_code != 200:
                    if time.time() - start_time >= 30:
                        print("During get {}th accountID's matchID(gameID), problem has been raised".format(i))
                        break
                    time.sleep(5)
                    match_url = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/'\
                            + accountIdList[i]  +'?season=13' + '&api_key=' + api_key
                    r = requests.get(match_url)

                if r.status_code == 200:
                    match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])

            gameIdList = match_info_df['gameId'].tolist()

    # 게임 ID 리스트 저장
            fileName = './Additional Game ID List/'+tier+division+str(page)+'gameIdList'
            with open(fileName, 'wb') as f:
                pickle.dump(gameIdList, f)
            print(tier, division, page, "game ID list saved")
            print("# matches:", len(gameIdList), "\n")

GOLD III start
summonerNameList 205
During get 14th summonerName's accountID, problem has been raised
During get 49th summonerName's accountID, problem has been raised
During get 80th summonerName's accountID, problem has been raised
During get 93th summonerName's accountID, problem has been raised
During get 95th summonerName's accountID, problem has been raised
During get 151th summonerName's accountID, problem has been raised
During get 152th summonerName's accountID, problem has been raised
During get 169th summonerName's accountID, problem has been raised
During get 199th summonerName's accountID, problem has been raised
accountIdList 196
During get 131th accountID's matchID(gameID), problem has been raised
During get 132th accountID's matchID(gameID), problem has been raised
GOLD III 2 game ID list saved
# matches: 19116 

GOLD II start
summonerNameList 205
During get 22th summonerName's accountID, problem has been raised
During get 29th summonerName's accountID, problem has been

During get 75th summonerName's accountID, problem has been raised
During get 96th summonerName's accountID, problem has been raised
During get 111th summonerName's accountID, problem has been raised
During get 113th summonerName's accountID, problem has been raised
During get 127th summonerName's accountID, problem has been raised
During get 138th summonerName's accountID, problem has been raised
During get 191th summonerName's accountID, problem has been raised
During get 196th summonerName's accountID, problem has been raised
accountIdList 190
During get 79th accountID's matchID(gameID), problem has been raised
During get 180th accountID's matchID(gameID), problem has been raised
During get 181th accountID's matchID(gameID), problem has been raised
GOLD II 4 game ID list saved
# matches: 16568 

GOLD I start
summonerNameList 205
During get 8th summonerName's accountID, problem has been raised
During get 23th summonerName's accountID, problem has been raised
During get 25th summonerNa